In [ ]:
import sys
# INSERT PATH TO THE PROJECT HERE: 
sys.path.insert(0,'/mnt/c/inz/inz/')
from wutsat.fun import config as cf
from wutsat.fun import sat_fun as s
from wutsat.fun import mat_fun as mat
from wutsat.fun import plotting_presets as pltpr
from wutsat.fun import cv_fun
from math import radians as to_rad
from skyfield.api import Topos, load
from IPython.display import clear_output
from IPython.display import Image
import numpy as np
from wutsat.fun.parse import InitVariables

In [4]:
%%time
generate_photos = 1
data_range = range(17)

# Calculate corresponding photos
for i in tmp:
    #Load saved results:
    v = InitVariables(i)
    # Calculate positions half footprint distance away:
    sat_positions = s.get_sat_positions(v.t,cf.pwsat2_tle_path,v.TLE)
    distance = s.satellite_basic(sat_positions[3][0], sat_positions[2][0])[1]/2
    distances = [s.satellite_basic(sat_positions[3][i], sat_positions[2][i])[1]/2 for i in range(len(sat_positions[0]))]
   
    
    extent, other_extents, res = cv_fun.find_corresponding_photos(v.projected_set_path, v.source_img, v.results_path, save_photos=1, save_path=v.dataset_path, calculate_only=False)
    area_def, files, area_extent = mat.rwdata(v.dataset_path, 'sat_data.pkl', 'r', None)
    
#     #Calculate coordinates:
    ext_lats, ext_lons = s.get_extent_in_coordinates(extent[2],area_def[extent[3][5]],files,v.composite)
    if generate_photos:
        pltpr.draw_poins_on_map(ext_lats, ext_lons, v.res_im1,800)

#     #Generates calculated up coordinates again to plot photo:
    photo_extent, photo_points = s.get_photo_extent_and_points(area_def[extent[3][5]],files,v.composite)
    if generate_photos:
        pltpr.plot_coastlines_on_map(v.composite_plot, files, photo_extent, [ext_lats, ext_lons], v.res_im2, 200)

#     #Generate overall footprint,detected area polygon, :
    photo_extent = [-30,45,0,90]

    foot_print_whole = s.calculate_sets_of_footprints(sat_positions[0],sat_positions[1],distances,sat_positions[3])

    detected_area = [[ext_lats.copy()], [ext_lons.copy()]]
    detected_area[0][0].append(ext_lats[0].copy())
    detected_area[1][0].append(ext_lons[0].copy())
    
    points = [[lat,lon] for lat,lon in zip(ext_lats, ext_lons)]
    t1,t1points, t1polygons,debug = s.get_timeframe_of_sat_positions(points, cf.pwsat2_tle_path, v.TLE, v.t, distances[0],plot=False)
    results = mat.calculate_results(v.t,t1,v)
    
    mat.save_results(results,v.dataset_path,other_data=[res[0],extent[3][4]])
    print(results)


    plt_pols = [foot_print_whole,t1polygons,detected_area]
    plt_lines = [[[sat_positions[0]],[sat_positions[1]]],[[debug[0][0]],[debug[0][1]]]]
    plt_points = [t1points]

    if generate_photos:
        pltpr.draw_polygons_on_map(plt_pols,plt_lines,plt_points,v.composite_plot, files, photo_extent, v.res_im3)



    list_of_images = [extent[3][2],extent[3][3],extent[3][4],v.res_im1,v.res_im2, v.res_im3]

    if generate_photos:
        pltpr.save_collage(list_of_images,v.res_final,'/mnt/c/inz/tmp/',images_per_row=3)
    cf.end_sound()
    
   

CPU times: user 1.28 s, sys: 1.41 s, total: 2.69 s
Wall time: 404 ms
